# Assorted helpers

In [ ]:
#| default_exp helpers

In [ ]:
#| export

from web3 import Web3, constants


In [ ]:
#| export

def normalize_address(address: str) -> str: return Web3.to_checksum_address(address.lower())

ADDRESS_ZERO = constants.ADDRESS_ZERO

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()